In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point
import numpy as np
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import h3
from shapely import wkt
from shapely.geometry import box, Polygon
import folium
import os

# Load data
all_lts_df = pd.read_csv("/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/Trento_all_lts.csv")
all_lts_df['geometry'] = all_lts_df['geometry'].apply(wkt.loads)
gdf_nodes = pd.read_csv("/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/Trento_gdf_nodes.csv", index_col=0)
gdf_nodes['geometry'] = gdf_nodes['geometry'].apply(wkt.loads)

#Convert the DataFrame to a GeoDataFrame
all_lts = gpd.GeoDataFrame(all_lts_df, geometry='geometry')
all_lts.crs = "EPSG:32632"
all_lts_projected = all_lts.to_crs(epsg=4326)

nodes = gpd.GeoDataFrame(gdf_nodes, geometry='geometry')
nodes.crs = "EPSG:32632"
nodes_projected = nodes.to_crs(epsg=4326)

def classify_stress(row):
    if pd.notna(row['lts']):
        if row['lts'] in [1, 2]:
            return 'low'
        elif row['lts'] in [3, 4]:
            return 'high'
    return None 

all_lts_projected['type_stress'] = all_lts_projected.apply(classify_stress, axis=1)
nodes_projected['type_stress'] = nodes_projected.apply(classify_stress, axis=1)

G = nx.Graph()
for idx, row in nodes_projected.iterrows():
    G.add_node(idx, **row.to_dict())

for _, row in all_lts_projected.iterrows():
    u = row['u']
    v = row['v']
    G.add_edge(u, v, **row.to_dict())

base_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/images"
city_name = "Trento"
city_folder_path = os.path.join(base_path, city_name)

# Create the folder if it doesn't exist
if not os.path.exists(city_folder_path):
    os.makedirs(city_folder_path)

isolated_nodes = list(nx.isolates(G))
num_isolated_nodes = len(isolated_nodes)
G.remove_nodes_from(isolated_nodes)

# Choose a layout
pos = {node: (data['geometry'].x, data['geometry'].y) for node, data in G.nodes(data=True)}

# Plotting
plt.figure(figsize=(10, 8))

# Draw with different colors for low and high stress
node_color = ['green' if data['type_stress'] == 'low' else 'red' for node, data in G.nodes(data=True)]
edge_color = ['green' if data['type_stress'] == 'low' else 'red' for u, v, data in G.edges(data=True)]
nx.draw_networkx_nodes(G, pos, node_color=node_color, node_size=5, alpha=0.8)
nx.draw_networkx_edges(G, pos, edge_color=edge_color, alpha=0.5)
plt.title("Bike Network in Trento")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


# Create subgraphs based on 'type_stress'
low_stress_edges = [(u, v) for u, v, d in G.edges(data=True) if d['type_stress'] == 'low']
high_stress_edges = [(u, v) for u, v, d in G.edges(data=True) if d['type_stress'] == 'high']

low_stress_subgraph = G.edge_subgraph(low_stress_edges).copy()
high_stress_subgraph = G.edge_subgraph(high_stress_edges).copy()

# Set up the matplotlib figure with two subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

# Low stress subplot
plt.subplot(1, 2, 1)
nx.draw_networkx_nodes(low_stress_subgraph, pos, node_size=5, alpha=0.8)
nx.draw_networkx_edges(low_stress_subgraph, pos, alpha=0.5)
plt.title("Low Stress Bike Network in Trento")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# High stress subplot
plt.subplot(1, 2, 2)
nx.draw_networkx_nodes(high_stress_subgraph, pos, node_size=5, alpha=0.8)
nx.draw_networkx_edges(high_stress_subgraph, pos, alpha=0.5)
plt.title("High Stress Bike Network in Trento")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Show the plot
plt.show()

In [ ]:
# Function to retrieve population data from OpenStreetMap
def get_osm_population(city_name):
    query = f"""
    [out:xml][timeout:25];
    area[name="{city_name}"]->.searchArea;
    (
      node["population"](area.searchArea);
      way["population"](area.searchArea);
      relation["population"](area.searchArea);
    );
    out body;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.get(url, params={'data': query})
    root = ET.fromstring(response.content)
    # Extract population data
    for element in root.iter('tag'):
        if element.attrib['k'] == 'population':
            return element.attrib['v']
    return "Population data not found"

total_city_population = int(get_osm_population("Trento"))

In [ ]:
from shapely.ops import unary_union
print("Starting process to generate city boundary...")
print("Simplifying geometries...")
simplified_geometries = all_lts_projected.geometry.simplify(tolerance=0.001)
print("Performing unary union on simplified geometries...")
simplified_union = unary_union(simplified_geometries)
print("Applying buffer...")
city_boundary = simplified_union.buffer(0.005)
print("City boundary generation completed.")

In [ ]:
# with open("../data/trento_city_boundary.wkt", "w") as file:
#     file.write(str(city_boundary))

from shapely.wkt import loads

with open("../data/trento_city_boundary.wkt", "r") as file:
    city_boundary = loads(file.read())

geo_file_path = '../data/kontur_population_IT_20220630.gpkg'

pop = gpd.read_file(geo_file_path)
pop = pop.to_crs("EPSG:4326")
pop_gdf = pop[pop.geometry.intersects(city_boundary)]
print(pop_gdf)


In [ ]:
# Function to create a hexagonal grid within the city boundaries
def create_hex_grid_within_city_bounds(city_boundary, resolution):
    hexagons = h3.polyfill(city_boundary.__geo_interface__, resolution, geo_json_conformant=True)

    filtered_hexagons = []
    for h in hexagons:
        hex_polygon = Polygon(h3.h3_to_geo_boundary(h, geo_json=True))
        if hex_polygon.intersects(city_boundary):
            filtered_hexagons.append(hex_polygon)

    hex_grid = gpd.GeoDataFrame([{'geometry': hexagon} for hexagon in filtered_hexagons])
    hex_grid.crs = 'EPSG:4326'

    return hex_grid

hex_grid_within_city = create_hex_grid_within_city_bounds(city_boundary, 9)
hex_grid_projected = hex_grid_within_city.to_crs('EPSG: 32632')

# # Calculate area for each hexagon to distribute population
# hex_grid_projected['area'] = hex_grid_projected['geometry'].area
# # Distribute the city's population across the hexagons
# hex_grid_projected['estimated_population'] = (hex_grid_projected['area'] / hex_grid_projected['area'].sum()) * total_city_population
pop_gdf = pop_gdf.to_crs(hex_grid_projected.crs)

# Spatial join
joined_gdf = gpd.sjoin(hex_grid_projected, pop_gdf, how="left", predicate='intersects')

# Since one hexagon might intersect with multiple population polygons,
# we aggregate the population data for each hexagon.
# Here we sum the populations, but you might choose a different method
# depending on your specific requirements.
hex_grid_with_population = joined_gdf.groupby(joined_gdf.index).agg({'population': 'sum'})
hex_grid_projected = hex_grid_projected.merge(hex_grid_with_population, left_index=True, right_index=True)
hex_grid_projected['population'] = hex_grid_projected['population'].fillna(0)
hex_grid_projected = hex_grid_projected.to_crs('EPSG:4326')
projected_crs = 'EPSG:32632' 
all_lts_projected_crs = all_lts_projected.to_crs(projected_crs)

# Calculate centroids
centroids = all_lts_projected_crs.geometry.centroid
centroids = centroids.to_crs(epsg=4326)

# Get the center latitude and longitude for the map
center_lat, center_lon = centroids.iloc[0].y, centroids.iloc[0].x
hex_grid_geojson = hex_grid_projected.to_json()

# Function to retrieve building data 
def get_building_data(city_name):
    # Retrieve buildings from OSM within the city boundary
    buildings = ox.features_from_place(city_name, tags={'building': True})
    return buildings

# Retrieve building data for city
buildings = get_building_data("Trento, Italy")
buildings = buildings.to_crs(hex_grid_projected.crs)
hex_grid_projected.reset_index(inplace=True)
hex_grid_projected.rename(columns={'index': 'hex_index'}, inplace=True)
hex_grid_with_buildings = gpd.sjoin(hex_grid_projected, buildings, how='left', predicate='intersects')
building_counts = hex_grid_with_buildings.groupby('hex_index').size()
hex_grid_projected['building_count'] = hex_grid_projected['hex_index'].map(building_counts).fillna(0)

# Adjust population estimation based on building count: here => simple average of area-based and building-based estimates
# hex_grid_projected['adjusted_population'] = hex_grid_projected.apply(
#     lambda row: (row['population'] + row['building_count']) / 2, axis=1)
# assert 'adjusted_population' in hex_grid_projected.columns, "Adjusted population column not found"

hex_grid_geojson = hex_grid_projected.to_crs(epsg=4326).to_json()

m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

def color_function(feature):
    population = feature['properties']['population']
    
    high_population_threshold = total_city_population * 0.02  # 2% of total population
    medium_population_threshold = total_city_population * 0.01  # 1% of total population

    if population > high_population_threshold:
        return '#ff0000'  # Red 
    elif population > medium_population_threshold:
        return '#ffff00'  # Yellow 
    else:
        return '#00ff00'  # Green 

folium.GeoJson(
    hex_grid_geojson,
    name='Hexagonal Grid',
    style_function=lambda feature: {
        'fillColor': color_function(feature),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5,
    }
).add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)

m

In [ ]:
file_path = os.path.join(city_folder_path, 'hexagonal_grid_population.html')

m.save(file_path)

In [ ]:
# Spatial join between hexagons and edges
edges_gdf = gpd.GeoDataFrame(all_lts_projected[['geometry', 'u', 'v', 'type_stress']], geometry='geometry')
hex_edges = gpd.sjoin(hex_grid_projected, edges_gdf, how='left', predicate='intersects')
# Group edges by hexagon using a list for column names
hexagon_edges = hex_edges.groupby('hex_index')[['u', 'v']].apply(lambda x: list(zip(x['u'], x['v'])))
# Remove hexagons with only NaN values in their edge list
filtered_hexagon_edges = {hex_id: edges for hex_id, edges in hexagon_edges.items() if not all(np.isnan(edge).any() for edge in edges)}

# Filtering is done by checking 'type_stress' for 'low'
hexagon_edges_low_stress = hex_edges[hex_edges['type_stress'] == 'low'].groupby('hex_index')[['u', 'v']].apply(lambda x: list(zip(x['u'], x['v'])))

# Remove hexagons with only NaN values in their edge list
filtered_hexagon_edges_low_stress = {hex_id: edges for hex_id, edges in hexagon_edges_low_stress.items() if not all(np.isnan(edge).any() for edge in edges)}

In [ ]:
# Prova shortests path:
from scipy.spatial import cKDTree
import networkx as nx
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor
import json


def find_hexagon_centroid(hexagon_geometry):
    return hexagon_geometry.centroid

# Precompute node points
node_points = {node: Point(data['x'], data['y']) for node, data in G.nodes(data=True)}
node_tree = cKDTree([(point.x, point.y) for point in node_points.values()])

# Function to find the closest node to a given point
def find_closest_node(centroid):
    _, nearest_node_index = node_tree.query((centroid.x, centroid.y))
    return list(node_points.keys())[nearest_node_index]

# Finding representative nodes for each hexagon
representative_nodes = {}
for hex_id in filtered_hexagon_edges_low_stress:
    representative_node = find_closest_node(find_hexagon_centroid(hex_grid_projected.loc[hex_grid_projected['hex_index'] == hex_id, 'geometry'].iloc[0]))
    representative_nodes[hex_id] = representative_node

# Check if the low-stress subgraph is connected
is_connected = nx.is_connected(low_stress_subgraph)
print(f"Is the low-stress subgraph connected? {is_connected}")

# If not connected, find out how many connected components it has
connected_components = list(nx.connected_components(low_stress_subgraph)) if not is_connected else []

# Count representative nodes in the low-stress subgraph
existing_representative_nodes = {hex_id: node for hex_id, node in representative_nodes.items() if node in low_stress_subgraph}
num_existing_representative_nodes = len(existing_representative_nodes)
total_representative_nodes = len(representative_nodes)
print(f"Number of representative nodes in low-stress subgraph: {num_existing_representative_nodes} out of {total_representative_nodes}")

# Function to check if two nodes are in the same connected component
def in_same_component(node1, node2, components):
    for component in components:
        if node1 in component and node2 in component:
            return True
    return False

# Function to compute shortest paths in low-stress network
def compute_shortest_paths_low_stress(start_hex_id, connected_components):
    paths = {}
    start_node = representative_nodes[start_hex_id]
    for target_hex_id, target_node in representative_nodes.items():
        if start_hex_id != target_hex_id and in_same_component(start_node, target_node, connected_components):
            if nx.has_path(low_stress_subgraph, start_node, target_node):
                path_length = nx.dijkstra_path_length(low_stress_subgraph, source=start_node, target=target_node, weight='length')
                paths[target_hex_id] = path_length
            else:
                paths[target_hex_id] = None
    return start_hex_id, paths

# Start parallel computation of shortest low-stress paths
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(compute_shortest_paths_low_stress, hex_id, connected_components) for hex_id in representative_nodes]
    shortest_low_stress_paths = {future.result()[0]: future.result()[1] for future in futures}
    executor.shutdown()

# Save the shortest_low_stress_paths result to a JSON file
file_path = '../data/shortest_low_stress_paths.json'

with open(file_path, 'w') as file:
    json.dump(shortest_low_stress_paths, file)

shortest_low_stress_paths

In [ ]:
from scipy.spatial import cKDTree
import networkx as nx
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor

def find_hexagon_centroid(hexagon_geometry):
    return hexagon_geometry.centroid

print("Precomputing node geometries...")
node_points = {node: Point(data['x'], data['y']) for node, data in G.nodes(data=True)}

print("Creating a spatial index for node points...")
node_tree = cKDTree([(point.x, point.y) for point in node_points.values()])

def find_closest_node(centroid):
    _, nearest_node_index = node_tree.query((centroid.x, centroid.y))
    return list(node_points.keys())[nearest_node_index]

print("Finding the representative node for each hexagon...")
representative_nodes = {}
for hex_id in filtered_hexagon_edges:
    representative_node = find_closest_node(find_hexagon_centroid(hex_grid_projected.loc[hex_grid_projected['hex_index'] == hex_id, 'geometry'].iloc[0]))
    representative_nodes[hex_id] = representative_node
    print(f"Processed hexagon {hex_id}, found representative node: {representative_node}")

# Function to compute shortest paths in parallel
def compute_shortest_paths(start_hex_id):
    print(f"Computing shortest paths for hexagon {start_hex_id}...")
    paths = {}
    start_node = representative_nodes[start_hex_id]
    for target_hex_id, target_node in representative_nodes.items():
        if start_hex_id != target_hex_id and nx.has_path(G, start_node, target_node): 
            path_length = nx.dijkstra_path_length(G, source=start_node, target=target_node, weight='length')
            paths[target_hex_id] = path_length
        else:
            paths[target_hex_id] = None
    print(f"Completed shortest paths for hexagon {start_hex_id}")
    return start_hex_id, paths

print("Starting computation of shortest paths using parallel processing...")
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(compute_shortest_paths, hex_id) for hex_id in representative_nodes]
    shortest_paths = {future.result()[0]: future.result()[1] for future in futures}
    executor.shutdown()  

print("Completed computation of shortest paths.")
shortest_paths

# Save the result to a JSON file
file_path = '../data/shortest_paths.json'
with open(file_path, 'w') as file:
    json.dump(shortest_paths, file)
    print(f"Saved shortest paths to {file_path}")

In [ ]:
import json

# Path to your JSON files
file1_path = '../data/shortest_low_stress_paths.json'
file2_path = '../data/shortest_paths.json'

# Reading the first JSON file
with open(file1_path, 'r') as file:
    shortest_low_stress_paths = json.load(file)

# Reading the second JSON file
with open(file2_path, 'r') as file:
    shortest_paths = json.load(file)

In [ ]:
N = 150  # Number of paths to extract
first_n_paths = {k: shortest_paths[k] for k in list(shortest_paths)[:N]}

In [ ]:
hexagon_connections = {}
detour_threshold = 1.5

for hex_id in shortest_paths:
    hexagon_connections[hex_id] = {}

    for target_hex_id in shortest_paths[hex_id]:
        baseline_path_length = shortest_paths[hex_id][target_hex_id]

        # Check if the hex_id and target_hex_id exist in shortest_low_stress_paths
        if hex_id in shortest_low_stress_paths and target_hex_id in shortest_low_stress_paths[hex_id]:
            low_stress_path_length = shortest_low_stress_paths[hex_id][target_hex_id]
        else:
            low_stress_path_length = None  # Assign None if not found

        # Determine if a connection should be made based on the path lengths
        if low_stress_path_length is not None and (baseline_path_length is None or low_stress_path_length <= baseline_path_length * detour_threshold):
            hexagon_connections[hex_id][target_hex_id] = True
        else:
            hexagon_connections[hex_id][target_hex_id] = False

# hexagon_connections now contains information about which hexagons are connected via low-stress paths
# hexagon_connections

# Initialize counters
true_count = 0
false_count = 0

# Iterate through the dictionary
for hex_id in hexagon_connections:
    for target_hex_id in hexagon_connections[hex_id]:
        if hexagon_connections[hex_id][target_hex_id]:
            true_count += 1
        else:
            false_count += 1

print("Number of True connections:", true_count)
print("Number of False connections:", false_count)

In [ ]:
def query_osm_building_types(city_name, building_types):
    all_destinations = []
    for building_type in building_types:
        try:
            query = {'building': building_type}
            buildings = ox.features_from_place(city_name, tags=query)

            for idx, building in buildings.iterrows():
                destination = {
                    'name': building.get('name', 'unknown'),
                    'type': building_type,
                    'coordinates': (building.geometry.centroid.y, building.geometry.centroid.x)
                }
                all_destinations.append(destination)
        except Exception as e:
            print(f"Error querying building type '{building_type}': {e}")

    return all_destinations

def query_osm_leisure_types(city_name, leisure_types):
    all_destinations = []
    for leisure_type in leisure_types:
        try:
            query = {'leisure': leisure_type}
            leisure_facilities = ox.features_from_place(city_name, tags=query)

            for idx, facility in leisure_facilities.iterrows():
                destination = {
                    'name': facility.get('name', 'unknown'),
                    'type': leisure_type,
                    'coordinates': (facility.geometry.centroid.y, facility.geometry.centroid.x)
                }
                all_destinations.append(destination)
        except Exception as e:
            print(f"Error querying leisure type '{leisure_type}': {e}")

    return all_destinations

def query_osm_shop_types(city_name, shop_types):
    all_destinations = []
    for shop_type in shop_types:
        try:
            query = {'shop': shop_type}
            shops = ox.features_from_place(city_name, tags=query)

            for idx, shop in shops.iterrows():
                destination = {
                    'name': shop.get('name', 'unknown'),
                    'type': shop_type,
                    'coordinates': (shop.geometry.centroid.y, shop.geometry.centroid.x)
                }
                all_destinations.append(destination)
        except Exception as e:
            print(f"Error querying shop type '{shop_type}': {e}")

    return all_destinations

def query_osm_bus_stops(city_name):
    all_bus_stops = []
    try:
        query = {'highway': 'bus_stop'}
        bus_stops = ox.features_from_place(city_name, tags=query)

        for idx, bus_stop in bus_stops.iterrows():
            name = str(bus_stop.get('name', 'Unknown')).replace('/', '_').replace('"', '')
            stop = {
                'name': name,
                'type': "bus_stop",
                'coordinates': (bus_stop.geometry.centroid.y, bus_stop.geometry.centroid.x)
            }
            all_bus_stops.append(stop)
    except Exception as e:
        print(f"Error querying bus stops: {e}")

    return all_bus_stops

# Use
city_name = "Trento, Italy"
shop_types = ['mall']
leisure_types = ['sports_centre', 'stadium', 'park', 'fitness_centre']
building_types = ['sports_centre', 'train_station', 'supermarket', 'bus_station', 'fitness_centre', 'mall', 'retail', 'shop']
destinations_shop = query_osm_shop_types(city_name, shop_types)
destinations_lei = query_osm_leisure_types(city_name, leisure_types)
destinations_build = query_osm_building_types(city_name, building_types)
#bus_stops = query_osm_bus_stops(city_name)

combined_destinations = destinations_shop + destinations_lei + destinations_build #+ bus_stops
unique_coordinates = set()
dsnt = []
for destination in combined_destinations:
    coords = destination['coordinates']
    if coords not in unique_coordinates:
        dsnt.append(destination)
        unique_coordinates.add(coords)
        

In [ ]:
import json

# Load the destinations category data
with open('destinations.json') as f:
    destinations_data = json.load(f)

# Extracting destination types
destination_types = []
for category in destinations_data['categories']:
    for dest_type in category['types']:
        destination_types.append(dest_type['type_name'])

def query_osm_for_destinations(city_name, dest_types):
    all_destinations = []
    for dest_type in dest_types:
        try:
            # Initially query with 'amenity' tag
            destinations = ox.features_from_place(city_name, tags={'amenity': dest_type})
            if not destinations.empty:
                for idx, row in destinations.iterrows():
                    all_destinations.append({
                        'name': row.get('name', 'Unknown'),
                        'type': dest_type,
                        'coordinates': (row.geometry.centroid.y, row.geometry.centroid.x)
                    })
            else:
                print(f"No data returned for destination type: {dest_type}")
        except Exception as e:
            print(f"Error querying {dest_type}: {e}")

    return all_destinations

list_destinations = query_osm_for_destinations(city_name, destination_types)
list_destinations = dsnt + list_destinations
list_destinations

# Initialize an empty set to track unique types
unique_types = set()

# Iterate through the dsnt list and collect unique types
for destination in list_destinations:
    unique_types.add(destination['type'])

# # Print the distinct types
# print("Distinct Types in list destinations:")
# for t in unique_types:
#     print(t)

# Associate destinations with hexagons
points_geometry = [Point(coord[1], coord[0]) for coord in [destination['coordinates'] for destination in list_destinations]]
gdf_destinations = gpd.GeoDataFrame(list_destinations, geometry=points_geometry)
gdf_destinations.crs = 'EPSG:4326'

# Ensuring CRS match
gdf_destinations = gdf_destinations.to_crs(hex_grid_projected.crs)
#print(gdf_destinations)
# Spatial join
joined_df = gpd.sjoin(hex_grid_projected, gdf_destinations, how='left', predicate='contains')
print(joined_df)

# Check for NaN values in the specified columns
nan_check = joined_df[['index_right', 'name', 'type', 'coordinates']].isna().all(axis=1).sum()
#print(f"\nNumber of rows where 'index_right', 'name', 'type', 'coordinates' are all NaN: {nan_check}")

# Checking for non-NaN values
not_nan_check = joined_df[['index_right', 'name', 'type', 'coordinates']].dropna().shape[0]
#print(f"\nNumber of rows with valid data in 'index_right', 'name', 'type', 'coordinates': {not_nan_check}")

In [ ]:
# Function to check if a row represents a destination hexagon
def is_destination(row):
    return not pd.isna(row.get('index_right')) and not pd.isna(row.get('name')) and not pd.isna(row.get('type')) and not pd.isna(row.get('coordinates'))

# Identify destination hexagons
joined_df['centroid'] = joined_df.geometry.centroid
destination_hexagons = joined_df[joined_df.apply(is_destination, axis=1)]

# Reproject to a suitable projected CRS
#projected_crs = 'EPSG:32632'  
joined_df = joined_df.to_crs(projected_crs)


print("Sample Destination Hexagons:")
print(destination_hexagons.head())
#print(joined_df.head())

In [ ]:
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
import networkx as nx
from scipy.spatial import cKDTree

# Cache for CRS transformers
transformers_cache = {}

def get_transformer(point_crs, graph_crs):
    if (point_crs, graph_crs) not in transformers_cache:
        point_proj = pyproj.CRS(point_crs)
        graph_proj = pyproj.CRS(graph_crs)
        transformers_cache[(point_crs, graph_crs)] = pyproj.Transformer.from_crs(point_proj, graph_proj, always_xy=True).transform
    return transformers_cache[(point_crs, graph_crs)]

def transform_point_to_graph_crs(point, point_crs, graph_crs):
    project = get_transformer(point_crs, graph_crs)
    return transform(project, point)

def build_spatial_index(G):
    points = [(data['x'], data['y']) for _, data in G.nodes(data=True)]
    return cKDTree(points), points

def nearest_node(G, point, spatial_index, point_crs='EPSG:32632', graph_crs='EPSG:4326'):
    transformed_point = transform_point_to_graph_crs(point, point_crs, graph_crs)
    nearest_idx = spatial_index.query(transformed_point.coords[0])[1]
    nearest_node_id = list(G.nodes)[nearest_idx]
    return nearest_node_id

def dijkstra_distance(G, source_geometry, target_geometry, spatial_index, point_crs='EPSG:32632', graph_crs='EPSG:4326'):
    source_node = nearest_node(G, source_geometry, spatial_index, point_crs, graph_crs)
    target_node = nearest_node(G, target_geometry, spatial_index, point_crs, graph_crs)
    try:
        distance = nx.dijkstra_path_length(G, source=source_node, target=target_node, weight='length')
    except nx.NetworkXNoPath:
        distance = float('inf')
    return distance

# Pre-processing step
spatial_index, _ = build_spatial_index(G)

# Main loop for calculating accessible destinations
hexagon_destinations_1 = {}
for index, hexagon in joined_df[:400].iterrows():
    print(f"Processing hexagon at index {index}")
    accessible_destinations = []
    for dest_index, destination in destination_hexagons.iterrows():
        if 'centroid' in hexagon and 'centroid' in destination:
            distance = dijkstra_distance(G, hexagon['centroid'], destination['centroid'], spatial_index)
            if distance <= 2000:
                accessible_destinations.append((destination['name'], destination['type'], destination['geometry'], dest_index))

    hexagon_destinations_1[(index, hexagon['geometry'])] = {
        'total_number_of_destinations': len(accessible_destinations),
        'destinations': accessible_destinations
    }
    print(f"Processed hexagon at index {index}")

print("Processing complete. Hexagon destinations calculated.")


In [ ]:
hexagon_destinations_1

In [ ]:
hexagon_destinations_2 = {}
for index, hexagon in joined_df[400:801].iterrows():
    print(f"Processing hexagon at index {index}")
    accessible_destinations = []
    for dest_index, destination in destination_hexagons.iterrows():
        if 'centroid' in hexagon and 'centroid' in destination:
            distance = dijkstra_distance(G, hexagon['centroid'], destination['centroid'], spatial_index)
            if distance <= 2000:
                accessible_destinations.append((destination['name'], destination['type'], destination['geometry'], dest_index))

    hexagon_destinations_2[(index, hexagon['geometry'])] = {
        'total_number_of_destinations': len(accessible_destinations),
        'destinations': accessible_destinations
    }
    print(f"Processed hexagon at index {index}")

In [ ]:
hexagon_destinations_2

In [ ]:
hexagon_destinations_3 = {}
for index, hexagon in joined_df[801:].iterrows():
    print(f"Processing hexagon at index {index}")
    accessible_destinations = []
    for dest_index, destination in destination_hexagons.iterrows():
        if 'centroid' in hexagon and 'centroid' in destination:
            distance = dijkstra_distance(G, hexagon['centroid'], destination['centroid'], spatial_index)
            if distance <= 2000:
                accessible_destinations.append((destination['name'], destination['type'], destination['geometry'], dest_index))

    hexagon_destinations_3[(index, hexagon['geometry'])] = {
        'total_number_of_destinations': len(accessible_destinations),
        'destinations': accessible_destinations
    }
    print(f"Processed hexagon at index {index}")

In [ ]:
hexagon_destinations_3

In [ ]:
hexagon_destinations = {**hexagon_destinations_1, **hexagon_destinations_2,**hexagon_destinations_3}


In [ ]:
hexagon_destinations

In [ ]:
def save_dict_to_file(dict_data, file_path):
    with open(file_path, 'w') as file:
        for key, value in dict_data.items():
            file.write(f"{key}: {value}\n")
        print(f"Data saved to {file_path}")

file_path = "../data/hexagon_destinations_shortest_path.txt" 
save_dict_to_file(hexagon_destinations, file_path)


In [ ]:
spatial_index, _ = build_spatial_index(low_stress_subgraph)

# Second loop for G_low_stress graph
for index, hexagon in joined_df.iterrows():
    accessible_destinations_low = []
    for dest_index, destination in destination_hexagons.iterrows():
        # Ensure that centroid exists for both hexagon and destination
        if 'centroid' in hexagon and 'centroid' in destination:
            distance = dijkstra_distance(low_stress_subgraph, hexagon['centroid'], destination['centroid'], spatial_index)

            if distance <= 2000: 
                accessible_destinations_low.append((destination['name'], destination['type'], destination['geometry'], dest_index))

    # Update the existing hexagon_destinations dictionary
    hexagon_destinations[(index, hexagon['geometry'])].update({
        'total_number_of_destinations_low': len(accessible_destinations_low),
        'destinations_low': accessible_destinations_low
    })
    print(f"Processed hexagon at index {index}")

hexagon_destinations

In [ ]:
# Load scoring processes
with open('bna_scoring.json') as file:
    scoring_data = json.load(file)

# Convert the JSON data into more accessible structures
destination_weights = {category['category_name']: category['weight'] for category in destinations_data['categories']}
type_weights = {dest_type['type_name']: dest_type['weight'] for category in destinations_data['categories'] for dest_type in category['types']}
scoring_processes = {process['process']: process['criteria'] for process in scoring_data['scoring_process']}

# Function to calculate score for a single destination type
def calculate_score_for_type(destinations, scoring_process):
    criteria = scoring_processes[scoring_process]
    score = 0
    for i, destination in enumerate(destinations):
        if i < len(criteria):
            score += criteria[i]['points']
    return score

# Score each hexagon
for hexagon, hex_data in hexagon_destinations.items():
    hex_scores = {}
    for category in destinations_data['categories']:
        cat_score = 0
        for dest_type in category['types']:
            type_name = dest_type['type_name']
            scoring_proc = dest_type['scoring_process']
            destinations = [d for d in hex_data['destinations'] if d[1] == type_name]
            type_score = calculate_score_for_type(destinations, scoring_proc)
            weighted_type_score = type_score * type_weights[type_name] / 100
            cat_score += weighted_type_score

        # Apply category weight
        weighted_cat_score = cat_score * destination_weights[category['category_name']] / 100
        hex_scores[category['category_name']] = weighted_cat_score

    hexagon_destinations[hexagon]['scores'] = hex_scores

for hexagon in hexagon_destinations:
    total_score = sum(hexagon_destinations[hexagon]['scores'].values())
    hexagon_destinations[hexagon]['total_score'] = total_score

# Iterate over hexagons to calculate total scores and overall score
overall_score = 0
total_population = sum(joined_df['population'])

for index, hexagon in joined_df.iterrows():
    hex_population = hexagon['population']
    hex_key = (index, hexagon['geometry'])
    hex_data = hexagon_destinations.get(hex_key, {})
    hex_score = hex_data.get('total_score', 0)  # Get the total score for the hexagon

    # Weight the hexagon's score by its population and add to overall score
    weighted_score = hex_score * (hex_population / total_population)
    overall_score += weighted_score

# Print the final overall score
print(f"Overall Score: {overall_score}")

In [ ]:
# Extracting necessary data
data = []
for (index, polygon), details in hexagon_destinations.items():
    total_score = details['total_score']
    data.append((index, polygon, total_score))

# Creating DataFrame
df = pd.DataFrame(data, columns=['index', 'geometry', 'total_score'])

# Setting the index
df.set_index('index', inplace=True)

# Converting to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Setting up the correct CRS if needed
gdf.crs = "EPSG:4326"

# The result is a GeoDataFrame named hex_totalscore
hex_totalscore = gdf
hex_totalscore

# Specify the file path and name
file_path = '../data/hex_totalscore.shp'

# Save the GeoDataFrame as a Shapefile
hex_totalscore.to_file(file_path)

# Read the Shapefile into a GeoDataFrame
loaded_gdf = gpd.read_file(file_path)

# Set the 'index' column as the index of the GeoDataFrame
loaded_gdf.set_index('index', inplace=True)

# Display the loaded GeoDataFrame to check its contents
print(loaded_gdf)



In [ ]:
import seaborn as sns

loaded_gdf['rounded_score'] = loaded_gdf['total_scor'].round().astype(int)

# Group by the rounded_score and count the number of hexagons per score
hexagon_count = loaded_gdf.groupby('rounded_score').size().reset_index(name='nr_hexagons')
descriptive_stats = loaded_gdf['rounded_score'].describe()

# Using seaborn to improve the visualization
plt.figure(figsize=(14, 8))
sns.barplot(x='rounded_score', y='nr_hexagons', data=hexagon_count, palette='viridis')

# Adding titles and labels
plt.title('Distribution of Hexagons for Rounded Scores', fontsize=16)
plt.xlabel('Rounded Total Score', fontsize=14)
plt.ylabel('Number of Hexagons', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
sns.despine()

file_path = os.path.join(city_folder_path, 'distribution_totalscores_plot.png')
plt.savefig(file_path)
# Show the plot
plt.show()
print(descriptive_stats)



In [ ]:
import branca.colormap as cm

hex_dest_data = [{'geometry': hex_geom, 'total_score': hex_data['total_score']} 
                 for (_, hex_geom), hex_data in hexagon_destinations.items()]

hex_dest_gdf = gpd.GeoDataFrame(hex_dest_data, crs='EPSG:32632')

hex_dest_gdf_projected = hex_dest_gdf.to_crs('EPSG:4326')
center_lat = hex_dest_gdf_projected.geometry.apply(lambda geom: geom.centroid.y).mean()
center_lon = hex_dest_gdf_projected.geometry.apply(lambda geom: geom.centroid.x).mean()

# Create a base folium map
map_da = folium.Map(location=[center_lat, center_lon], zoom_start=11.2)


# Function to determine the color of a hexagon based on its total_score
def get_hexagon_color(total_score):
    # You can adjust the color scheme based on your preference and scoring scale
    if total_score > 20:
        return 'green'
    elif total_score > 10:
        return 'orange'
    elif total_score > 5:
        return 'red'
    else:
        return 'darkred'

legend_colormap = cm.LinearColormap(
    colors=['darkred', 'red', 'orange', 'green'],
    index=[0, 5, 10, 20],
    vmin=0,
    vmax=20,
    caption='Total Score'
)

legend_colormap.add_to(map_da)

for _, row in hex_dest_gdf_projected.iterrows():
    color = get_hexagon_color(row['total_score'])
    folium.GeoJson(
        row['geometry'],
        style_function=lambda _, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    ).add_to(map_da)

file_path = os.path.join(city_folder_path, 'bna_score_map.html')

# Assuming 'accident_map' is a Folium Map object
map_da.save(file_path)

map_da